In [6]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import faiss
import numpy as np
import os

from pillow_heif import register_heif_opener

register_heif_opener()

In [ ]:
#Define a function that normalizes embeddings and add them to the index
def add_vector_to_index(embedding, index):
    #convert embedding to numpy
    vector = embedding.detach().cpu().numpy()
    #Convert to float32 numpy
    vector = np.float32(vector)
    #Normalize vector: important to avoid wrong results when searching
    faiss.normalize_L2(vector)
    #Add to index
    index.add(vector)


    #Populate the images variable with all the images in the dataset folder
images = []
for root, dirs, files in os.walk('../gallery/photos/'):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.heic'):
            images.append(root + '/' + file)

print('Number of images :', len(images))

#load the model and processor
device = torch.device("cpu")
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small').to(device)

#Create Faiss index using FlatL2 type with 384 dimensions as this
#is the number of dimensions of the features
index = faiss.IndexFlatL2(384)

import time
t0 = time.time()
for image_path in images:
    img = Image.open(image_path).convert('RGB')
    with torch.no_grad():
        inputs = processor(images=img, return_tensors="pt").to(device)
        outputs = model(**inputs)
    features = outputs.last_hidden_state
    add_vector_to_index( features.mean(dim=1), index)

print('Extraction done in :', time.time()-t0)

#Store the index locally
faiss.write_index(index,"vector.index")

Number of images : 480
Extraction done in : 206.97534608840942


In [4]:
images = []
for root, dirs, files in os.walk('../gallery/photos/'):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.heic'):
            images.append(root + '/' + file)

print('Number of images :', len(images))

Number of images : 480


In [5]:
#pickel the images list
import pickle
with open('images.pkl', 'wb') as f:
    pickle.dump(images, f)

In [4]:
import faiss
import numpy as np
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image

#input image
image = Image.open(r"C:\Users\garra\Desktop\img_2835.heic").convert('RGB')

#Load the model and processor
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small').to(device)

#Extract the features
with torch.no_grad():
    inputs = processor(images=image, return_tensors="pt").to(device)
    outputs = model(**inputs)

#Normalize the features before search
embeddings = outputs.last_hidden_state
embeddings = embeddings.mean(dim=1)
vector = embeddings.detach().cpu().numpy()
vector = np.float32(vector)
faiss.normalize_L2(vector)

#Read the index file and perform search of top-3 images
index = faiss.read_index("vector.index")
d,i = index.search(vector,3)
print('distances:', d, 'indexes:', i)

distances: [[0.         0.07601181 0.08240125]] indexes: [[44 48 47]]


In [8]:
index = faiss.read_index("../vector.index")

In [30]:
with open('../images.pkl', 'rb') as f:
    images = pickle.load(f)

print(len(images))

475


In [31]:
images

['./gallery/photos/2023\\05\\img_2728.heic',
 './gallery/photos/2023\\05\\img_2729.heic',
 './gallery/photos/2023\\05\\img_2730.heic',
 './gallery/photos/2023\\05\\img_2731.heic',
 './gallery/photos/2023\\05\\img_2740.heic',
 './gallery/photos/2023\\05\\img_2752.png',
 './gallery/photos/2023\\05\\img_2753.jpg',
 './gallery/photos/2023\\05\\img_2755.heic',
 './gallery/photos/2023\\05\\img_2757-collage.jpg',
 './gallery/photos/2023\\05\\img_2757.heic',
 './gallery/photos/2023\\05\\img_2758.heic',
 './gallery/photos/2023\\05\\img_2759.heic',
 './gallery/photos/2023\\05\\img_2760.heic',
 './gallery/photos/2023\\05\\img_2761.heic',
 './gallery/photos/2023\\06\\bb36cb3e-0a26-4419-b41a-063c9c72aa90.jpg',
 './gallery/photos/2023\\06\\img_2775.heic',
 './gallery/photos/2023\\06\\img_2777.png',
 './gallery/photos/2023\\06\\img_2780.jpg',
 './gallery/photos/2023\\06\\img_2781.heic',
 './gallery/photos/2023\\06\\img_2784.jpg',
 './gallery/photos/2023\\06\\img_2785.heic',
 './gallery/photos/2023\\0

In [9]:
dir(index)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'add',
 'add_c',
 'add_with_ids',
 'add_with_ids_c',
 'assign',
 'assign_c',
 'check_compatible_for_merge',
 'code_size',
 'codes',
 'compute_distance_subset',
 'compute_residual',
 'compute_residual_n',
 'd',
 'get_CodePacker',
 'get_FlatCodesDistanceComputer',
 'get_distance_computer',
 'get_xb',
 'is_trained',
 'merge_from',
 'metric_arg',
 'metric_type',
 'ntotal',
 'range_search',
 'range_search_c',
 'reconstruct',
 'reconstruct_batch',
 'reconstruct_batch_c',
 'reconstruct_c',
 'reconstruct_n',
 'reconstruct_n_c',
 'remove_ids',
 'remove_ids_c',
 'reset',
 'sa_c

In [10]:
help(index.add)

Help on method replacement_add in module faiss.class_wrappers:

replacement_add(x) method of faiss.swigfaiss_avx2.IndexFlat instance
    Adds vectors to the index.
    The index must be trained before vectors can be added to it.
    The vectors are implicitly numbered in sequence. When `n` vectors are
    added to the index, they are given ids `ntotal`, `ntotal + 1`, ..., `ntotal + n - 1`.
    
    Parameters
    ----------
    x : array_like
        Query vectors, shape (n, d) where d is appropriate for the index.
        `dtype` must be float32.



In [5]:
top_image_paths = [images[i] for i in i[0]]  # top_indexes[0] contains the indexes of the top 3 images

# Display the images
for img_path in top_image_paths:
    img = Image.open(img_path)
    img.show()

In [11]:
index.ntotal

480

In [23]:
index.remove_ids(np.array([477]))

1

In [24]:
index.ntotal

477

In [25]:
a=[]

def test():
    a.append(1)

test()  
print(a)

[1]


In [32]:
import binascii
filepath="2e2f67616c6c6572792f70686f746f732f323032335c30355c696d675f323732392e68656963"
binascii.unhexlify(filepath.encode('utf-8')).decode()

'./gallery/photos/2023\\05\\img_2729.heic'